In [11]:
import numpy as np
import matplotlib.pyplot as plt
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import freqopttest.glo as glo
import scipy.stats as stats
import sys
from collections import defaultdict
import os
import pandas as pd
import time
import pickle

In [52]:
import itertools
window = [1, 2, 3, 6, 8, 10]
shift = [1, 2, 3]
for r in itertools.product(window, shift): print(r[1] , r[0])

1 1
2 1
3 1
1 2
2 2
3 2
1 3
2 3
3 3
1 6
2 6
3 6
1 8
2 8
3 8
1 10
2 10
3 10


In [50]:
windows = [1, 2, 3, 6, 8]
shifts = [3]
for r in itertools.product(windows, shifts):
    print(windows[0], shift[1])

1 2
1 2
1 2
1 2
1 2


In [12]:


def simulate_null_spectral(weights, n_simulate=1000, seed=275):
    """
    weights: chi-square weights (for the infinite weigted sum of chi squares)
    Return the values of MMD^2 (NOT n*MMD^2) simulated from the null distribution by
    the spectral method.
    """
    # draw at most block_size values at a time
    block_size = 400
    D = chi2_weights.shape[0]  # len(weights)
    mmds = np.zeros(n_simulate)
    from_ind = 0

    with util.NumpySeedContext(seed=seed):
        while from_ind < n_simulate:
            to_draw = min(block_size, n_simulate - from_ind)
            # draw chi^2 random variables.
            chi2 = np.random.randn(D, to_draw) ** 2
            # an array of length to_draw
            sim_mmds = 2.0 * weights.dot(chi2 - 1.0)
            # store
            end_ind = from_ind + to_draw
            mmds[from_ind:end_ind] = sim_mmds
            from_ind = end_ind
    return mmds


def chi_square_weights_H0(k, X):
    """
    Return a numpy array of the weights to be used as the weights in the
    weighted sum of chi-squares for the null distribution of MMD^2.
    - k: a Kernel
    - X: n x d number array of n data points
    """
    n = X.shape[0]
    # Gram matrix
    K = k.eval(X, X)
    # centring matrix. Not the most efficient way.
    H = np.eye(n) - np.ones((n, n)) / float(n)
    HKH = H.dot(K).dot(H)
    # https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.eigvals.html
    evals = np.linalg.eigvals(HKH)
    evals = np.real(evals)
    # sort in decreasing order
    evals = -np.sort(-evals)
    weights = evals / float(n) ** 2
    return weights


In [36]:
t7 ='/media/ak/T71'
experimentFiles = os.path.join(t7,'August11th2022Experiments')
ExperimentOne = os.path.join(experimentFiles, 'ExperimentOne')
mfdfaDataFrames = os.path.join(experimentFiles, 'mfdfaDataFrames')
# '/media/ak/T71/August11th2022Experiments/mfdfaDataFrames'  # Input Files
LinearMMDOutputFiles = '/media/ak/T71/August11th2022Experiments/ExperimentOne/LinearMMDOutputFiles'  # Location to Save Files
symbol = 'DU1'
bar_choice = 'calendar'
variable = 'alpha'

In [44]:
symbolMFDFADataFrames = os.path.join(ExperimentOne, symbol)
test = os.path.join(mfdfaDataFrames, 'FB1_calendar_alpha.pkl')
pd.read_pickle(test)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.000000,0.000000,5.395859,0.000000,0.000000,0.000000,0.000000,9.964629,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,5.401244,0.000000,0.000000,0.000000,0.000000,9.969986,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,5.406898,0.000000,0.000000,0.000000,0.000000,9.975203,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,5.412737,0.000000,0.000000,0.000000,0.000000,9.980149,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,5.418604,0.000000,0.000000,0.000000,0.000000,9.984659,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.038065,-0.001908,0.003573,0.116332,-0.000780,0.004164,0.012034,0.002931,0.006161,0.004187,...,0.004055,0.003460,0.003366,0.002610,0.003530,0.006200,0.002250,0.000562,0.000565,0.012281
65,0.037341,-0.002118,0.003405,0.115607,-0.001035,0.003632,0.011195,0.002549,0.005745,0.003850,...,0.003659,0.003026,0.002957,0.002494,0.003063,0.005788,0.001985,0.000331,0.000045,0.011572
66,0.036792,-0.002283,0.003326,0.114891,-0.001237,0.003287,0.010601,0.002278,0.005465,0.003636,...,0.003425,0.002696,0.002722,0.002406,0.002728,0.005508,0.001846,0.000145,-0.000275,0.011061
67,0.036368,-0.002417,0.003309,0.114180,-0.001401,0.003074,0.010186,0.002086,0.005281,0.003507,...,0.003305,0.002439,0.002608,0.002338,0.002488,0.005322,0.001795,-0.000008,-0.000454,0.010693


In [35]:
os.listdir(mfdfaDataFrames)

['FB1', 'JB1', 'TestMMDOutputFiles', 'US1', 'DU1', 'QuadMMDOutputFiles']

In [26]:
file_name = [f for f in os.listdir(symbolMFDFADataFrames) if str(symbol) in f][0]
print(file_name)

DU1_12tick_mfdfa.pkl


In [32]:



file_loc = os.path.join(symbolMFDFADataFrames, file_name)
unpickled_Df = pd.read_pickle(file_loc)
# len = unpickled_Df.shape[1]
# print(len)
# mmd_train_test_results = defaultdict(dict)
# mmd_test_results_one_dict = defaultdict(dict)
# mmd_test_results_two_dict = defaultdict(dict)
unpickled_Df['tick']['alpha']

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan, 0.47833557, 0.45782376, 0.44464197, 0.43814965,
       0.43762938, 0.44195493, 0.44955079, 0.45841508, 0.46621636,
       0.47048749, 0.46891723, 0.45970142, 0.44187592, 0.41553408,
       0.38184875, 0.34287341, 0.30116703, 0.2593442 , 0.21967026,
       0.18379514, 0.15266281, 0.12657401, 0.10534158, 0.08847255,
       0.07532921, 0.06524681, 0.0576061 , 0.05186928, 0.04759103,
       0.04441489, 0.0420626 , 0.04032087, 0.03902855, 0.03806518,
       0.03734147, 0.03679169, 0.03636775, 0.03603459])

In [ ]:
if __name__ == '__main__':
    shift = 2
    window = 8  # slide
    start = time.time()
    for start_point in range(0, (len - window - shift)):
        end_point = start_point + shift

        X = np.array(unpickled_Df.iloc[:, start_point:end_point])
        Y = np.array(unpickled_Df.iloc[:, end_point + shift:end_point + 2 * shift])
        Z = np.array(unpickled_Df.iloc[:, start_point + window:end_point + window])

        tr_data = data.TSTData(X, Y)
        test_data_one = data.TSTData(X, Z)  # data to train the model
        test_data_two = data.TSTData(Y, Z)
        # training dictionary results
        tr, te = tr_data.split_tr_te(tr_proportion=0.95, seed=10)  # is this necessary?!

        xtr, ytr = tr.xy()
        xytr = tr.stack_xy()
        sig2 = util.meddistance(xytr, subsample=1000)
        k = kernel.KGauss(sig2)
        mean, var = tst.QuadMMDTest.h1_mean_var(xtr, ytr, k, is_var_computed=True)
        Kx = k.eval(xtr, xtr)
        Ky = k.eval(ytr, ytr)
        Kxy = k.eval(xtr, ytr)
        mean_gram, var_gram = tst.QuadMMDTest.h1_mean_var_gram(Kx, Ky, Kxy, k, True)
        chi2_weights = chi_square_weights_H0(k, xytr)
        sim_mmds = simulate_null_spectral(chi2_weights, n_simulate=2000)
        # choose the best parameter and perform a test with permutations
        med = util.meddistance(tr.stack_xy(), 1000)
        list_gwidth = np.hstack(((med ** 2) * (2.0 ** np.linspace(-4, 4, 20))))
        list_gwidth.sort()
        list_kernels = [kernel.KGauss(gw2) for gw2 in list_gwidth]

        # grid search to choose the best Gaussian width
        besti, powers = tst.QuadMMDTest.grid_search_kernel(tr, list_kernels, alpha=0.05)
        # perform test
        best_ker = kernel.KGauss(list_kernels[besti])
        mmd_train_test_results[start_point]['perm_mmds1'] = tst.QuadMMDTest.permutation_list_mmd2(xtr, ytr, k,
                                                                                                  n_permute=2000)
        mmd_train_test_results[start_point]['test_stat'] = tst.QuadMMDTest.permutation_list_mmd2(xtr, ytr, k,
                                                                                                  n_permute=2000)

        mmd_train_test_results[start_point]['chi2_weights'] = chi2_weights
        mmd_train_test_results[start_point]['sim_mmds'] = sim_mmds
        mmd_train_test_results[start_point]['sig2'] = sig2
        mmd_train_test_results[start_point]['Kxy'] = k.eval(xtr, ytr)
        mmd_train_test_results[start_point]['mean'] = mean
        mmd_train_test_results[start_point]['var'] = var
        mmd_train_test_results[start_point]['Kxx'] = k.eval(xtr, xtr)
        mmd_train_test_results[start_point]['Kyy'] = k.eval(ytr, ytr)
        mmd_train_test_results[start_point]['mean_gram'] = mean_gram
        mmd_train_test_results[start_point]['var_gram'] = var_gram
        mmd_train_test_results[start_point]['med'] = util.meddistance(tr.stack_xy(), 1000)
        mmd_train_test_results[start_point]['list_gwidth'] = list_gwidth.sort()
        mmd_train_test_results[start_point]['list_kernels'] = list_kernels
        mmd_train_test_results[start_point]['besti'] = besti
        mmd_train_test_results[start_point]['powers'] = powers
        mmd_train_test_results[start_point]['best_ker'] = best_ker

        alpha = 0.05
        mmd_test = tst.QuadMMDTest(best_ker, n_permute=2000, alpha=alpha)
        mmd_train_test_results[start_point]['XZ_test'] = mmd_test.perform_test(test_data_one)
        mmd_train_test_results[start_point]['YZ_test'] = mmd_test.perform_test(test_data_two)

        # testing the first data set
    pickle_out_dict_one = os.path.join(LinearMMDOutputFiles, "".join(
        (str(symbol) + "_" + str(variable) + "_shift_" + str(shift) + "_wind_" + str(window) + "_" + str(
            'quad_MMD_test') + ".pkl")))
    pickle.dump(mmd_train_test_results, open(pickle_out_dict_one, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

    end = time.time()
    print(end - start)

#%%
